In [1]:
import os
import re
import sys
from collections import Counter
from itertools import islice
from urllib.parse import urlparse, urlsplit, parse_qs, parse_qsl

import numpy as np
import parsel
import sklearn_crfsuite
from sklearn_crfsuite.metrics import flat_classification_report, sequence_accuracy_score
from sklearn.model_selection import cross_val_predict, GroupKFold
# from formasaurus.utils import download

sys.path.insert(0, '..')
from autopager.storage import Storage
from autopager.htmlutils import (get_link_text, get_text_around_selector_list,
                                 get_link_href, get_selector_root)
from autopager.utils import (
    get_domain, normalize_whitespaces, normalize, ngrams, tokenize, ngrams_wb, replace_digits
)
from autopager.evaluate import get_annotation_folds
from autopager.model import link_to_features, _num_tokens_feature, _elem_attr, AUTOPAGER_LIMITS

In [2]:
storage = Storage()
urls = [rec['Page URL'] for rec in storage.iter_records()]
X_raw, y = storage.get_Xy()
print("pages: {}  domains: {}".format(len(urls), len({get_domain(url) for url in urls})))

Not all links are matched {'<a>1</a>'}
Not all links are matched {'<a>6</a>'}
Not all links are matched {'<a onclick="display(\'grid\');">Mosaico</a>'}
Not all links are matched {'<a onclick="display(\'grid\');">Mosaico</a>'}
Not all links are matched {'<a class="btn prev disabled" type="backward"><i class="icon-chevron-left"> </i></a>'}
pages: 196  domains: 81


In [3]:
%%time
# XXX: these functions should be copy-pasted from autopager/model.py

def link_to_features(link):
    text = normalize(get_link_text(link))

    href = get_link_href(link)
    p = urlsplit(href)

    query_parsed = parse_qsl(p.query)
    query_param_names = [k.lower() for k, v in query_parsed]
    query_param_names_ngrams = list(ngrams_wb(
        " ".join([normalize(name) for name in query_param_names]), 3, 5, True
    ))

    elem = get_selector_root(link)
    elem_target = _elem_attr(elem, 'target')
    elem_rel = _elem_attr(elem, 'rel')

    # Classes of link itself and all its children.
    # It is common to have e.g. span elements with fontawesome
    # arrow icon classes inside <a> links.
    self_and_children_classes = ' '.join(link.xpath(".//@class").extract())
    parent_classes = ' '.join(link.xpath('../@class').extract())
    css_classes = normalize(parent_classes + ' ' + self_and_children_classes)

    return {
        'bias': 3.0,
        'isdigit': text.isdigit(),
        'isalpha': text.isalpha(),
        'elem-target': elem_target,
        'elem-rel': elem_rel,
        'num-tokens%s' % _num_tokens_feature(text): 1.0,

        'text': list(islice(ngrams_wb(replace_digits(text), 2, 5),
                            0, AUTOPAGER_LIMITS.max_text_features)),
        'text-exact': replace_digits(text.strip()[:20].strip()),
        'class': list(islice(ngrams_wb(css_classes, 4, 5),
                             0, AUTOPAGER_LIMITS.max_css_features)),
        'query': query_param_names_ngrams,

        'path-has-page': 'page' in p.path.lower(),
        'path-has-pageXX': re.search(r'[/-](?:p|page\w?)/?\d+', p.path.lower()) is not None,
        'path-has-number': any(part.isdigit() for part in p.path.split('/')),

        'href-has-year': re.search('20\d\d', href) is not None,
    }


def page_to_features(xseq):
    features = [link_to_features(a) for a in xseq]
    around = get_text_around_selector_list(xseq, max_length=15)

    # weight is less than 1 because there is a lot of duplicate information
    # in these ngrams and so we want to regularize them stronger
    # (as if they are a single feature, not many features)
    k = 0.2
    for feat, (before, after) in zip(features, around):
        feat['text-before'] = {n: k for n in list(ngrams_wb(normalize(before), 5, 5))}
        feat['text-after'] = {n: k for n in list(ngrams_wb(normalize(after), 5, 5))}
    return features

X = [page_to_features(xseq) for xseq in X_raw]

CPU times: user 9.44 s, sys: 103 ms, total: 9.54 s
Wall time: 9.6 s


In [4]:
# X[60][12]

In [5]:
# TRAIN_SIZE = 80
# X_train, y_train = X[:TRAIN_SIZE], y[:TRAIN_SIZE]
# X_test, y_test = X[TRAIN_SIZE:], y[TRAIN_SIZE:]
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.001,
    c2=0.05,
    max_iterations=100,
    all_possible_transitions=True,
    verbose=False,
)
# crf.fit(X_train, y_train, X_test, y_test)

In [6]:
%%time
#folds = get_annotation_folds(urls, 6)
group_kfold = GroupKFold(6)
groups=[get_domain(url) for url in urls]
y_pred = cross_val_predict(crf, X, y, groups=groups, cv=group_kfold, n_jobs=-1)
print(flat_classification_report(y, y_pred, labels=['PAGE', 'NEXT', 'PREV'], digits=3))
print("Sequence accuracy: {:0.3f}".format(sequence_accuracy_score(y, y_pred)))

/home/ivan/Documentos/scrapinghub/dev/autocrawl_env/lib/python3.7/site-packages/sklearn/base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)
/home/ivan/Documentos/scrapinghub/dev/autocrawl_env/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass labels=['PAGE', 'NEXT', 'PREV'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


              precision    recall  f1-score   support

        PAGE      0.872     0.925     0.898      1178
        NEXT      0.903     0.781     0.837       155
        PREV      0.892     0.812     0.850       112

   micro avg      0.876     0.901     0.888      1445
   macro avg      0.889     0.839     0.862      1445
weighted avg      0.877     0.901     0.888      1445

Sequence accuracy: 0.612
CPU times: user 1min 1s, sys: 1.47 s, total: 1min 3s
Wall time: 1min 16s


In [7]:
crf.fit(X, y)
# crf.attributes_
crf.num_attributes_

11514

In [8]:
# [a for a in sorted(crf.attributes_) if a.startswith('id')]

## What are important features?

In [9]:
# XXX: weight for correlated features don't show their importance
# XXX: weights for features of different scale don't show their importance
# (e.g. coefficients to text-after and text-before features are high, but only
# because input is scaled down for these features)

def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))

print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(150))

print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-50:])

Top positive:
2.908590 PAGE     text:末页
2.908590 PAGE     text-exact:末页
2.897269 NEXT     class:next
2.674313 PAGE     isdigit
2.409984 NEXT     text:>
2.257835 O        elem-rel:nofollow
2.040691 PREV     text:<
1.928678 NEXT     text-exact:»
1.893480 PAGE     text-exact:首页
1.880481 PAGE     text-exact:X
1.846517 PAGE     text:首页
1.833171 NEXT     text-exact:>
1.763758 PREV     text-exact:«
1.687928 O        text:..
1.655565 PREV     text-exact:<
1.653429 PREV     class:prev
1.498988 O        text-exact:>>
1.455909 PREV     text:«
1.454248 NEXT     text:»
1.450213 PAGE     text:X
1.440575 NEXT     text:下一
1.440575 NEXT     text:下一页
1.440575 NEXT     text-exact:下一页
1.391984 PAGE     text:<<
1.384221 O        query:m
1.376550 PAGE     elem-rel:next
1.356580 PAGE     text:>|
1.337715 O        text-after:...
1.326451 O        href-has-year
1.326367 O        num-tokens>2
1.266192 PAGE     text-exact:XX
1.262865 PAGE     text-after:iconi
1.262865 PAGE     text-after:conin
1.262865 PAGE     

In [10]:
from collections import Counter

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Transitions:")
print_transitions(Counter(crf.transition_features_).most_common())

Transitions:
O      -> O       1.929150
PREV   -> PAGE    1.740921
PAGE   -> NEXT    1.687386
NEXT   -> PAGE    1.201963
PAGE   -> PREV    0.917556
PREV   -> NEXT    0.544653
O      -> PREV    0.039332
PAGE   -> PAGE    -0.262285
NEXT   -> O       -0.264870
NEXT   -> PREV    -0.604082
PREV   -> PREV    -0.839827
NEXT   -> NEXT    -1.175496
PAGE   -> O       -1.603856
O      -> NEXT    -1.770049
O      -> PAGE    -1.780110
PREV   -> O       -1.858937


## Let's check errors the model is making

In [11]:
folds = get_annotation_folds(urls, 6)
y_pred = cross_val_predict(crf, X, y, cv=folds, n_jobs=-1)
errors = np.asarray(y) != np.asarray(y_pred)
error_rows = np.asarray(list(storage.iter_records()))[errors]
error_links = np.asarray(X_raw)[errors]
error_y_pred = y_pred[errors]
error_y_true = np.asarray(y)[errors]

ModuleNotFoundError: No module named 'sklearn.cross_validation'

In [ ]:
for links, yseq_pred, yseq_true, row in zip(error_links, error_y_pred, error_y_true, error_rows):
    print(row['Page URL'])
    for label_correct, label_pred, link in zip(yseq_true, yseq_pred, links.extract()):
        if label_correct != label_pred:
            print("%4s %4s %s" % (label_correct, label_pred, link))
    print("\n")

## Unused code

In [ ]:
def _url_parts(url):
    p = urlsplit(url)
    args = parse_qsl(p.query)
    argnames = [name for name, value in args]
    return p.netloc, set(p.path.split('/')) | set(args) | set(argnames)

def url_distance(url1, url2):
    netloc1, parts1 = _url_parts(url1)
    netloc2, parts2 = _url_parts(url2)
    if netloc1 != netloc2:
        return 1.0
    if not parts1 and not parts2:
        return 0.0
    return 1 - len(parts1 & parts2) / len(parts1 | parts2)

#         dist = url_distance(url, href)
#         if dist == 0:
#             feat['url-distance=0'] = 1.0
#         elif dist == 1.0:
#             feat['url-distance=1'] = 1.0
#         else:
#             feat['url-distance=k'] = dist


url_distance('http://example.com/foo/345?page=2', 'http://example.com/foo/345?page=4')

In [ ]:
# def guess_page_number(link):
#     text = get_link_text(link).strip()
#     if text.isdigit():
#         return int(text)
#     return None

# def number_pattern2(pattern):
#     txt = re.sub('X+', 'X', pattern)
# #     txt = re.sub('C+', 'C', txt)
#     return txt

#     pagenums = [guess_page_number(a) for a in xseq]
# #     print(pagenums)
#     for i in range(1, len(xseq)):
#         if pagenums[i-1] is None or pagenums[i] is None:
#             features[i]['page-diff:None'] = 1.0
#         else:
#             diff = pagenums[i] - pagenums[i-1]
#             if diff == 1:
#                 features[i]['page-diff==1'] = 1.0
#             else:
#                 features[i]['page-diff<>1'] = 1.0